# 1. Setup and Initialization

## 1.1 Importing Necessary Packages

In [1]:
# System
import wandb
import os
import json
import gc

# Environment
from dotenv import load_dotenv
from huggingface_hub import login

# LLM packages
import numpy as np
import torch
from datasets import Dataset, DatasetDict, load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig,
    set_seed,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

# Set Seed
train_seed = 2002
set_seed(train_seed)

# 캐시 디렉토리 설정
DATA_CACHE_DIR = "/mnt/t7/.cache/huggingface/datasets"
MODEL_CACHE_DIR = "/mnt/t7/.cache/huggingface/models"

## 1.2 Logging into Hugging Face Hub and Weights & Biases

In [2]:
os.environ["WANDB_PROJECT"]="Graduate Project EXAONE-3.5-7.8B-Instruct_ft01"
wandb.login()

load_dotenv("/mnt/t7/dnn/llm_practicing/.env")
login(token= os.environ["HF_TOKEN"])

wandb: Currently logged in as: aeolian83 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


# 2. Loading and Preparing the Dataset

In [3]:
dataset_name = "aeolian83/PTT_wit_Latex_1"

dataset_dict = load_dataset(dataset_name, cache_dir=DATA_CACHE_DIR)

In [4]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['term', 'english', 'korean'],
        num_rows: 1432
    })
})

In [5]:
shuffled_train = dataset_dict["train"].shuffle(seed=42)

dataset_dict = DatasetDict({
    "train": shuffled_train,
    # 다른 split도 있으면 추가 (예: "validation": dataset["validation"])
})

# 3. Hyperparameter Configuration

In [6]:
# Configring Base Model Load 
model_id = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"
device_map = {"": 0}

# Configuring Quantization
load_in_4bit = True
bnb_4bit_compute_dtype = torch.bfloat16
bnb_4bit_quant_type = "nf4"
bnb_4bit_use_double_quant = True

#Defining Training Arguments
checkpoint_dir = "/mnt/t7/dnn/paper_translator2/test/checkpoint/exaone_3.5_7.8b_instruct_ft02"
output_dir = checkpoint_dir
per_device_train_batch_size = 1
gradient_accumulation_steps = 2
optim = "paged_adamw_32bit"
report_to="wandb"
save_strategy="epoch"
num_train_epochs = 4
logging_steps = 20
# eval_steps=100,
learning_rate = 2e-4
max_grad_norm = 0.3
warmup_ratio = 0.03
lr_scheduler_type = "cosine"
bf16 = True
group_by_length = True

# Configuring Lora
lora_r = 64
lora_alpha = 16
lora_dropout=0.1
target_modules='all-linear'
bias="none"
task_type="CAUSAL_LM"

# Configuring tokenizer
padding_side = "left"
response_template = "korean:"

# 4. Loading the Base Model for QLoRA

## 4.1 Loading the Model with QLoRA Configuration


In [7]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=load_in_4bit,
    bnb_4bit_compute_dtype=bnb_4bit_compute_dtype,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_use_double_quant=bnb_4bit_use_double_quant,
)

In [8]:
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, device_map=device_map, cache_dir=MODEL_CACHE_DIR, low_cpu_mem_usage=True, trust_remote_code=True,)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

## 4.2 Loading the Tokenizer and Setting up Data Collator

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=MODEL_CACHE_DIR)

tokenizer.padding_side = padding_side
response_template = response_template

data_collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

# 5. Training the Model
## 5.1 Defining Training Arguments

In [10]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    num_train_epochs=num_train_epochs,
    save_strategy=save_strategy,
    logging_steps=logging_steps,
    # eval_steps=eval_steps,
    report_to = report_to,
    learning_rate=learning_rate,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    seed=train_seed
)

## 5.2 Defining PEFT Lora Configuration and Formatting Function

In [11]:
# LoRA Config
peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=target_modules,
    bias=bias,
    task_type=task_type,
)

In [12]:
# 1) system_prompt: 교수님 역할만
system_prompt = "You are a professor specializing in Physics, proficient in both Korean and English. Your task is to translate English physics content into Korean, adhering to specific guidelines."

# 2) user_prompt: 가이드라인과 실제 입력을 함께
user_prompt_template = """<translation guideline>
1. CRITICAL: All technical terms, MUST be translated using the format: Korean term(English term). Example: 적대적 훈련(adversarial training).
2. For acronyms, use the following format: Korean full term(English full term, acronym). Example: 계층적으로 조직된 경량 다중 탐지 시스템(hierarchically organized light-weight multiple detector system, HOLMES).
3. Maintain an academic tone and ensure technical accuracy in your translation.
4. Produce natural-sounding Korean translation while accurately conveying the original meaning.
5. Change all letters within parentheses in Korean sentences to lowercase.
6. Ensure consistency in terminology and parenthetical translation throughout the text.
7. When translating equations or mathematical expressions, maintain the standard notation used in Korean academic physics papers.
8. Mathematical expressions written in LaTeX syntax must be displayed exactly as they are, without any modifications.
</translation guideline>

## Example Output
korean: 앙상블 학습(context of ensemble learning)에서 적응형 신경 프레임워크(adaptive neural frameworks)의 개발은 다양한 벤치마크 데이터셋(benchmark datasets)에서 광범위한 실험 결과로 입증된 바와 같이 심층 신경망(deep neural networks)의 성능을 크게 향상시킵니다. 이러한 적응형 신경 프레임워크(adaptive neural frameworks)를 활용함으로써 연구자들은 특징을 지능적으로 융합하여 더 차별화되고 효과적인 표현을 생성할 수 있으며, 이에 따라 모델의 일반화 능력을 향상시킬 수 있습니다. 결과적으로, 적응형 신경 프레임워크(adaptive neural frameworks)는 전통적인 특징 융합 기법(traditional feature fusion techniques)을 능가할 뿐만 아니라 이미지 분류(image classification), 객체 탐지(object detection), 자연어 처리(natural language processing, NLP), 그래프 기반 학습(graph-based learning) 작업을 포함한 여러 도메인에서 광범위한 적용 가능성을 보여줍니다.

## Output Format
korean: Sentences using with proper parenthetical translation.

Note: Provide only the Korean translation as output. Do not include the original English sentence.

Input: {english}
"""

In [13]:
# Formatting function
def formatting_func(example):
    formatted_texts = []
    for eng, kor in zip(example["english"], example["korean"]):
        # SYSTEM 메시지
        convo = f"[SYSTEM]\n{system_prompt}\n\n"
        # USER 메시지: guideline + 실제 입력
        user_prompt = user_prompt_template.format(english=eng)
        convo   += f"[USER]\n{user_prompt}\n"
        # ASSISTANT 메시지: 정답 + EOS
        convo   += f"[ASSISTANT]\nkorean: {kor}{tokenizer.eos_token}"
        formatted_texts.append(convo)
    return formatted_texts

## 5.3 Setting up Trainer with SFTTrainer

In [14]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    peft_config=peft_config,
    args=training_arguments,
    train_dataset=dataset_dict["train"],
    formatting_func=formatting_func,
    data_collator=data_collator,
)

/tmp/ipykernel_385346/468261088.py:1: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/1432 [00:00<?, ? examples/s]

/home/aeolian83/anaconda3/envs/llm_for_p312/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [15]:
trainer.train()
trainer.save_model(os.path.join(training_arguments.output_dir, "last_checkpoint"))

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
20,0.441500
40,0.336300
60,0.338800
80,0.310700
100,0.289100
120,0.308800
140,0.276300
160,0.265600
180,0.267600
200,0.282600


## Model Merge and Upload

In [16]:
# del(trainer)
# gc.collect()
# gc.collect()
# torch.cuda.empty_cache()
# torch.cuda.ipc_collect()

In [2]:
model_id = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"
device_map = {"": 0}
checkpoint_dir = "/mnt/t7/dnn/paper_translator2/test/checkpoint/exaone_3.5_7.8b_instruct_ft02/last_checkpoint"


In [3]:
model = AutoModelForCausalLM.from_pretrained(
            model_id, 
            torch_dtype=torch.float16, 
            load_in_8bit=False, 
            device_map=device_map, 
            trust_remote_code=True, 
            cache_dir=MODEL_CACHE_DIR)

tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=MODEL_CACHE_DIR)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
peft_model = PeftModel.from_pretrained(model, checkpoint_dir)

In [5]:
model = peft_model.merge_and_unload()

In [6]:
# LGAI-EXAONE/EXAONE3.5_7.8B-Inst_translator02

model.push_to_hub('aeolian83/EXAONE3.5_7.8B-Inst_translator02')

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/839M [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/aeolian83/EXAONE3.5_7.8B-Inst_translator02/commit/0c83eb44e768deb75d6ae7585571828ad616e92b', commit_message='Upload ExaoneForCausalLM', commit_description='', oid='0c83eb44e768deb75d6ae7585571828ad616e92b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/aeolian83/EXAONE3.5_7.8B-Inst_translator02', endpoint='https://huggingface.co', repo_type='model', repo_id='aeolian83/EXAONE3.5_7.8B-Inst_translator02'), pr_revision=None, pr_num=None)